# Imports et installation

### Import GIT

In [ ]:
# Import git projet

!git clone https://github.com/deca1111/Anomaly_Detection_Comparison.git

# changement de folder
%cd /content/Anomaly_Detection_Comparison/

Cloning into 'INF8225-Projet'...
remote: Enumerating objects: 2067, done.
remote: Counting objects: 100% (2067/2067), done.
remote: Compressing objects: 100% (743/743), done.
remote: Total 2067 (delta 675), reused 2063 (delta 671), pack-reused 0
Receiving objects: 100% (2067/2067), 2.83 MiB | 8.27 MiB/s, done.
Resolving deltas: 100% (675/675), done.
/content/INF8225-Projet


### Installation modules

In [ ]:
# Instalation modules utiles
# ! pip install -r requirements.txt

! pip install tensorboardX
! pip install pyyaml


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.3 MB/s eta 0:00:00


In [ ]:
# Import utiles

from anomaly_detection.piad.train import Trainer
from anomaly_detection.piad.evaluate import evaluate

import torch
import torch.nn as nn

import numpy as np
import yaml

import time
import os
import sys

import PIL.Image

import urllib.request
import tarfile

import csv

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(DEVICE)

cuda


# Configuration

## Téléchargement Dataset NIH

In [ ]:
# URLs for the zip files
links = [
    'https://nihcc.box.com/shared/static/vfk49d74nhbxq3nqjg0900w5nvkorp5c.gz',
    'https://nihcc.box.com/shared/static/i28rlmbvmfjbl8p2n3ril0pptcmcu9d1.gz',
    'https://nihcc.box.com/shared/static/f1t00wrtdk94satdfb9olcolqx20z2jp.gz',
	'https://nihcc.box.com/shared/static/0aowwzs5lhjrceb3qp67ahp0rd1l1etg.gz',
    'https://nihcc.box.com/shared/static/v5e3goj22zr6h8tzualxfsqlqaygfbsn.gz',
	'https://nihcc.box.com/shared/static/asi7ikud9jwnkrnkj99jnpfkjdes7l6l.gz',
	'https://nihcc.box.com/shared/static/jn1b4mw4n6lnh74ovmcjb8y48h8xj07n.gz',
    'https://nihcc.box.com/shared/static/tvpxmn7qyrgl0w8wfh9kqfjskv6nmm1j.gz',
	'https://nihcc.box.com/shared/static/upyy3ml7qdumlgk2rfcvlb9k6gvqq2pj.gz',
	'https://nihcc.box.com/shared/static/l6nilvfa9cg3s28tqv1qc1olm3gnz54p.gz',
	'https://nihcc.box.com/shared/static/hhq8fkdgvcari67vfhs7ppg2w6ni4jze.gz',
	'https://nihcc.box.com/shared/static/ioqwiy20ihqwyr8pf4c24eazhh281pbu.gz'
]

rootZip = './data/data/nih_zip/'

os.makedirs(rootZip, exist_ok=True)

start_time = time.time()

for idx, link in enumerate(links):

  fileName = f"images_{idx+1}.tar.gz"
  filePath = os.path.join(rootZip, fileName)

  # Check si le fichier est déjà telechargé
  if os.path.isfile(filePath):
    print("\n"+fileName+' already downloaded')
    continue

  nbLeft = len(links) - idx - 1
  print(f"\nDownloading {fileName} - {idx+1}/{len(links)} : ")

  # Avec ligne de commande
  ! wget -q --show-progress -O $filePath $link

  # Avec lib python
  # urllib.request.urlretrieve(link, filePath)  # download the zip file

minutesElapsed = (time.time() - start_time) // 60
secondsElapsed = (time.time() - start_time) % 60
print(f"Téléchargement terminé en {minutesElapsed}m {secondsElapsed}s")


./data/data/nih_zip 100%[===================>]   1.87G  22.6MB/s    in 86s     

./data/data/nih_zip 100%[===================>]   3.68G  22.6MB/s    in 2m 49s  

./data/data/nih_zip 100%[===================>]   3.66G  22.5MB/s    in 2m 48s  

./data/data/nih_zip 100%[===================>]   3.58G  22.2MB/s    in 2m 49s  

./data/data/nih_zip 100%[===================>]   3.67G  22.3MB/s    in 2m 48s  

./data/data/nih_zip 100%[===================>]   3.71G  22.8MB/s    in 2m 50s  

./data/data/nih_zip 100%[===================>]   3.74G  22.0MB/s    in 2m 52s  

./data/data/nih_zip 100%[===================>]   3.74G  22.8MB/s    in 2m 52s  

./data/data/nih_zip 100%[===================>]   3.83G  22.9MB/s    in 2m 56s  

./data/data/nih_zip 100%[===================>]   3.89G  22.5MB/s    in 2m 59s  

./data/data/nih_zip 100%[===================>]   3.90G  22.8MB/s    in 2m 59s  

./data/data/nih_zip 100%[===================>]   2.71G  21.0MB/s    in 2m 8s   
Téléchargement terminé en 32

## Unzip dataset

In [ ]:
rootUnzip = './data/data/nih/'

os.makedirs(rootUnzip, exist_ok=True)

start_time = time.time()

for filename in os.listdir(rootZip): # loop through items in dir
    if filename.endswith(".gz"): # check for ".gz" extension
        print("Unzipping "+filename)
        filePath = os.path.join(rootZip, filename)

        # Avec ligne de commande
        ! tar -xzf $filePath -C $rootUnzip

        # Avec lib python
        # with tarfile.open(filePath,"r") as gz_ref:
        #     gz_ref.extractall(rootUnzip)

minutesElapsed = (time.time() - start_time) // 60
secondsElapsed = (time.time() - start_time) % 60
print(f"\nUnzip terminé en {minutesElapsed}m {secondsElapsed}s")

Unzipping images_1.tar.gz
Unzipping images_3.tar.gz
Unzipping images_12.tar.gz
Unzipping images_10.tar.gz
Unzipping images_9.tar.gz
Unzipping images_7.tar.gz
Unzipping images_2.tar.gz
Unzipping images_8.tar.gz
Unzipping images_4.tar.gz
Unzipping images_6.tar.gz
Unzipping images_11.tar.gz
Unzipping images_5.tar.gz

Unzip terminé en 6.0m 45.44375991821289s


## Liste d'images Subset

Nous allons utiliser seulement le subset, nous n'avons donc pas besoin de toutes les images.

Nous allons donc créer des lists d'images que nous voulons qui sont dans le subset.


In [ ]:
subset_root = "./folds/train_test_split/nih/subset"

anomaly_root = os.path.join(subset_root, "anomaly")
normal_root = os.path.join(subset_root, "normal")

!ls $anomaly_root
!ls $normal_root

test
test  train


In [ ]:
images_anomaly_test_file = os.path.join(anomaly_root, "test")
images_normal_test_file = os.path.join(normal_root, "test")
images_normal_train_file = os.path.join(normal_root, "train")

with open(images_anomaly_test_file) as f_in:
  list_images_anomaly_test = [filename.strip() for filename in f_in.readlines()]

with open(images_normal_test_file) as f_in:
  list_images_normal_test = [filename.strip() for filename in f_in.readlines()]

with open(images_normal_train_file) as f_in:
  list_images_normal_train = [filename.strip() for filename in f_in.readlines()]

print(f"Nombre d'images anomaly test : {len(list_images_anomaly_test)}")
print(f"Nombre d'images normal test : {len(list_images_normal_test)}")
print(f"Nombre d'images normal train : {len(list_images_normal_train)}")

print(f"Nombre total d'images : {len(list_images_anomaly_test) + len(list_images_normal_test) + len(list_images_normal_train)}")

# Conversion en set pour être sur qu'on a pas de doublons
set_images_anomaly_test = set(list_images_anomaly_test)
set_images_normal_test = set(list_images_normal_test)
set_images_normal_train = set(list_images_normal_train)

set_total_images_nonFold = set_images_anomaly_test.union(set_images_normal_test).union(set_images_normal_train)

print(f"Nombre total d'images : {len(set_total_images_nonFold)}")

Nombre d'images anomaly test : 667
Nombre d'images normal test : 677
Nombre d'images normal train : 4261
Nombre total d'images : 5605
Nombre total d'images : 5605


### Folds

In [ ]:
subset_folds_root = "/content/Anomaly_Detection_Comparison/folds/folds/nih/subset"

!ls $subset_folds_root

folds = os.listdir(subset_folds_root)
print(folds)

set_total_images_Folds = set()

for fold in folds:
  print(f"Fold {fold}")

  fold_root = os.path.join(subset_folds_root, fold)

  fold_anomaly_root = os.path.join(fold_root, "anomaly")
  fold_normal_root = os.path.join(fold_root, "normal")

  set_anomaly_images = set()
  set_normal_images = set()

  for filename in os.listdir(fold_anomaly_root):

    with open(os.path.join(fold_anomaly_root, filename)) as f_in:
      list_anomaly_images = [filename.strip() for filename in f_in.readlines()]
      set_anomaly_images = set_anomaly_images.union(set(list_anomaly_images))

      print(f"Nombre d'images anomaly {filename} : {len(list_anomaly_images)}")

  print(f"Nombre d'images anomaly : {len(set_anomaly_images)}")

  for filename in os.listdir(fold_normal_root):

    with open(os.path.join(fold_normal_root, filename)) as f_in:
      list_normal_images = [filename.strip() for filename in f_in.readlines()]
      set_normal_images = set_normal_images.union(set(list_normal_images))

      print(f"Nombre d'images normal {filename} : {len(list_normal_images)}")

  print(f"Nombre d'images normal : {len(set_normal_images)}")

  set_total_images_Folds = set_total_images_Folds.union(set_anomaly_images).union(set_normal_images)

  print(f"Nombre total d'images après le fold {fold} : {len(set_total_images_Folds)}\n")

print(f"Nombre total d'images : {len(set_total_images_Folds)}")




0  1  2
['1', '0', '2']
Fold 1
Nombre d'images anomaly test : 857
Nombre d'images anomaly : 857
Nombre d'images normal train : 4261
Nombre d'images normal val : 849
Nombre d'images normal test : 849
Nombre d'images normal : 5110
Nombre total d'images après le fold 1 : 5967

Fold 0
Nombre d'images anomaly test : 857
Nombre d'images anomaly : 857
Nombre d'images normal train : 4261
Nombre d'images normal val : 849
Nombre d'images normal test : 849
Nombre d'images normal : 5110
Nombre total d'images après le fold 0 : 5967

Fold 2
Nombre d'images anomaly test : 857
Nombre d'images anomaly : 857
Nombre d'images normal train : 4261
Nombre d'images normal val : 849
Nombre d'images normal test : 849
Nombre d'images normal : 5110
Nombre total d'images après le fold 2 : 5967

Nombre total d'images : 5967


## Union folds & Non folds

In [ ]:
all_images_subset = set_total_images_nonFold.union(set_total_images_Folds)

print(f"Nombre total d'images : {len(all_images_subset)}")

Nombre total d'images : 7311


## Rezing subset

In [ ]:
print(f"Début du resizing de {len(all_images_subset)} images : \n")

inputRootImages = "/content/Anomaly_Detection_Comparison/data/data/nih/images"

outputRootImages = "/content/Anomaly_Detection_Comparison/data/data/nih/images_subset_300"

os.makedirs(outputRootImages, exist_ok=True)

for idx, image_name in enumerate(all_images_subset):
  if idx%100 == 0 :
    print(f"{idx}/{len(all_images_subset)}")

  image_path = os.path.join(inputRootImages, image_name)

  image = PIL.Image.open(image_path)
  image = image.convert('L')
  image = image.resize((300, 300), PIL.Image.BILINEAR)

  output_path = os.path.join(outputRootImages, image_name)
  image.save(output_path)

Début du resizing de 7311 images : 

0/7311
100/7311
200/7311
300/7311
400/7311
500/7311
600/7311
700/7311
800/7311
900/7311
1000/7311
1100/7311
1200/7311
1300/7311
1400/7311
1500/7311
1600/7311
1700/7311
1800/7311
1900/7311
2000/7311
2100/7311
2200/7311
2300/7311
2400/7311
2500/7311
2600/7311
2700/7311
2800/7311
2900/7311
3000/7311
3100/7311
3200/7311
3300/7311
3400/7311
3500/7311
3600/7311
3700/7311
3800/7311
3900/7311
4000/7311
4100/7311
4200/7311
4300/7311
4400/7311
4500/7311
4600/7311
4700/7311
4800/7311
4900/7311
5000/7311
5100/7311
5200/7311
5300/7311
5400/7311
5500/7311
5600/7311
5700/7311
5800/7311
5900/7311
6000/7311
6100/7311
6200/7311
6300/7311
6400/7311
6500/7311
6600/7311
6700/7311
6800/7311
6900/7311
7000/7311
7100/7311
7200/7311
7300/7311


## Zipping subset

In [ ]:
subsetRoot = "/content/Anomaly_Detection_Comparison/data/data/nih/images_subset_300"

outputPath = "/content/Anomaly_Detection_Comparison/data/data/nih/images_subset_300.zip"

!zip -r $outputPath $subsetRoot

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  adding: content/INF8225-Projet/data/data/nih/images_subset_300/00010481_028.png (deflated 0%)
  adding: content/INF8225-Projet/data/data/nih/images_subset_300/00003308_002.png (deflated 0%)
  adding: content/INF8225-Projet/data/data/nih/images_subset_300/00024368_000.png (deflated 0%)
  adding: content/INF8225-Projet/data/data/nih/images_subset_300/00024270_001.png (deflated 0%)
  adding: content/INF8225-Projet/data/data/nih/images_subset_300/00026278_000.png (deflated 0%)
  adding: content/INF8225-Projet/data/data/nih/images_subset_300/00030334_001.png (deflated 0%)
  adding: content/INF8225-Projet/data/data/nih/images_subset_300/00014327_000.png (deflated 0%)
  adding: content/INF8225-Projet/data/data/nih/images_subset_300/00021341_005.png (stored 0%)
  adding: content/INF8225-Projet/data/data/nih/images_subset_300/00009574_010.png (deflated 0%)
  adding: content/INF8225-Projet/data/data/nih/images_subset

## Copy in drive

In [ ]:
# Ajout du drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
inputFiles = "/content/Anomaly_Detection_Comparison/data/data/nih/images_subset_300.zip"
outputFiles = "/content/drive/MyDrive/Polytechnique/INF8225/Projet/images_subset_300.zip"

!cp $inputFiles $outputFiles

In [ ]:
# Put list of images into csv file
outputFile = "/content/Anomaly_Detection_Comparison/data/data/nih/images_subset_300.txt"

with open(outputFile, "w") as f_out:

  for idx, image_name in enumerate(all_images_subset):
    if idx%100 == 0 :
      print(f"{idx}/{len(all_images_subset)}")

    f_out.write(f"{image_name}\n")

0/7311
100/7311
200/7311
300/7311
400/7311
500/7311
600/7311
700/7311
800/7311
900/7311
1000/7311
1100/7311
1200/7311
1300/7311
1400/7311
1500/7311
1600/7311
1700/7311
1800/7311
1900/7311
2000/7311
2100/7311
2200/7311
2300/7311
2400/7311
2500/7311
2600/7311
2700/7311
2800/7311
2900/7311
3000/7311
3100/7311
3200/7311
3300/7311
3400/7311
3500/7311
3600/7311
3700/7311
3800/7311
3900/7311
4000/7311
4100/7311
4200/7311
4300/7311
4400/7311
4500/7311
4600/7311
4700/7311
4800/7311
4900/7311
5000/7311
5100/7311
5200/7311
5300/7311
5400/7311
5500/7311
5600/7311
5700/7311
5800/7311
5900/7311
6000/7311
6100/7311
6200/7311
6300/7311
6400/7311
6500/7311
6600/7311
6700/7311
6800/7311
6900/7311
7000/7311
7100/7311
7200/7311
7300/7311
